# <center>练习二——线性分类</center>

In [68]:
# 导入相关的库
import pandas as pd
import numpy as np

In [69]:
# 1. 数据加载
names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 
         'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 
         'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class']

data_path = "data/breast-cancer-wisconsin.data"
data = pd.read_csv(data_path, names=names)

# 2 数据预处理
# 2.1 数据清洗：去除含缺失值的样本
data = data.replace(to_replace="?", value=np.nan)  
data = data.dropna()

# 2.2 将特征列和标签列转换为数值类型
data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
data = data.dropna() 

x = data.iloc[:, 1:10].values.astype(np.float64)  
y = data["Class"].values.astype(int) 
y = np.where(y == 4, 1, 0)


In [70]:
# 2.3 数据集划分
def train_test_split_manual(X, y, test_size=0.25, random_state=2025):
    """手动实现训练集和测试集的划分"""
    if random_state is not None:
        np.random.seed(random_state)
    
    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)
    
    # 生成随机索引
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

In [71]:
# 2.4 特征标准化
class StandardScaler:
    # TODO
    # 实现特征标准化
    # 标准化公式: z = (x - mean) / std

    def __init__(self):
        self.mean_ = None
        self.std_ = None
    
    def fit(self, X):
        """计算均值和标准差"""
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)

    def transform(self, X):
        """使用均值和标准差进行标准化"""
        return (X - self.mean_) / self.std_
    
    def fit_transform(self, X):
        """拟合并转换"""
        self.fit(X)
        return self.transform(X)

## 逻辑回归

### 模型形式
$h_\theta(x) = \frac{1}{1 + e^{-\theta x}}$

* $x$：输入特征向量
* $\theta$：模型参数
* $h_\theta(x)$：预测结果（属于正类的概率）


$$
\text{class} =
\begin{cases}
1, & \text{if } h_\theta(x) \ge 0.5 \\
0, & \text{if } h_\theta(x) < 0.5
\end{cases}
$$

### 损失函数：

$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m}\Big[y^{(i)} \log(h_\theta(x^{(i)})) + (1 - y^{(i)}) \log(1 - h_\theta(x^{(i)}))\Big]$


### 优化：梯度下降

$\theta := \theta - \alpha \frac{\partial J(\theta)}{\partial \theta}$

其中 $\alpha$ 为学习率

In [ ]:
# 3. 实现逻辑回归
class LogisticRegression:
    """
    参数:
        learning_rate: 学习率
        n_iterations: 迭代次数
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
    
    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        # sigmoid(z) = 1 / (1 + e^(-z))
        return 1 / (1 + np.exp(-z))

    def compute(self, X: np.ndarray, y: np.ndarray):
        m=X.shape[0]

        z=np.dot(X,self.weights)+self.bias
        A=self.sigmoid(z)
        cost= -1/m * np.sum(y*np.log(A)+(1-y)*np.log(1-A))

        dz=A-y
        dw=(1.0/m)*np.dot(X.T,dz)
        db=(1.0/m)*np.sum(dz)

        return dw,db,cost
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        # TODO: 训练模型
        # 使用批量梯度下降法（BatchGradientDescent，BGD）优化权重和偏置
            
        # 提示:
        # 1. 初始化权重和偏置
        # 2. 进行n_iterations次迭代
        # 3. 每次迭代:
        #     step1. 计算预测值
        #     step2. 计算梯度
        #     step3. 更新参数
        self.weights=np.zeros(X.shape[1],dtype=float)
        self.bias=0.0

        for i in range(self.n_iterations):
            dw,db,cost=self.compute(X,y)
            self.weights=self.weights - self.learning_rate*dw
            self.bias=self.bias - self.learning_rate*db


        
    
    def predict(self, X: np.ndarray, threshold: float = 0.5) -> np.ndarray:
        # TODO: 预测
        # 提示:
        # 1. 进行线性运算
        # 2. 进行sigmoid计算
        # 3. 将结果与阈值进行比较
        z=np.dot(X,self.weights)+self.bias
        A=self.sigmoid(z)
        y_pred=(A>=threshold).astype(int)
        return y_pred


In [73]:
def get_metrics(y_true, y_pred):
    """
        获得评测指标
    """
    def recall_score(y_true, y_pred):
        """
        计算召回率
        召回率 = TP / (TP + FN)
        """
        TP = np.sum((y_true == 1) & (y_pred == 1))
        FN = np.sum((y_true == 1) & (y_pred == 0))
        return TP / (TP + FN) if (TP + FN) > 0 else 0.0

    def precision_score(y_true, y_pred):
        """精确率 = TP / (TP + FP)"""
        TP = np.sum((y_true == 1) & (y_pred == 1))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        return TP / (TP + FP) if (TP + FP) > 0 else 0.0
        
    def accuracy_score(y_true, y_pred):
        """准确率 = (TP + TN) / 总数"""
        return np.mean(y_true == y_pred)


    def confusion_matrix(y_true, y_pred):
        """混淆矩阵"""
        TN = np.sum((y_true == 0) & (y_pred == 0))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        FN = np.sum((y_true == 1) & (y_pred == 0))
        TP = np.sum((y_true == 1) & (y_pred == 1))
        
        return np.array([[TN, FP], [FN, TP]])
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    return recall, precision, accuracy, cm
    

In [ ]:
if __name__=='__main__':
    print("=" * 60)
    print("乳腺癌诊断预测 - 逻辑回归实现")
    print("=" * 60)
    
    # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split_manual(x, y, test_size=0.25, random_state=22)
    
    print(f"\n数据集信息:")
    print(f"训练集样本数: {X_train.shape[0]}")
    print(f"测试集样本数: {X_test.shape[0]}")
    print(f"特征数: {X_train.shape[1]}")
    
    # 特征标准化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    print("\n开始训练模型...")
    # 训练逻辑回归模型
    model = LogisticRegression(learning_rate=0.01, n_iterations=1000)
    model.fit(X_train, y_train)
    
    # 预测
    y_pred = model.predict(X_test)
    
    # 评估
    print("\n" + "=" * 60)
    print("模型评估结果")
    print("=" * 60)
    
    recall, precision, accuracy, cm = get_metrics(y_test, y_pred)
    
    print(f"\n召回率 (Recall):    {recall:.4f} ({recall*100:.2f}%)")
    print(f"精确率 (Precision): {precision:.4f} ({precision*100:.2f}%)")
    print(f"准确率 (Accuracy):  {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    print(f"\n混淆矩阵:")
    print(f"              预测负类  预测正类")
    print(f"实际负类        {cm[0,0]:4d}     {cm[0,1]:4d}")
    print(f"实际正类        {cm[1,0]:4d}     {cm[1,1]:4d}")

乳腺癌诊断预测 - 逻辑回归实现

数据集信息:
训练集样本数: 513
测试集样本数: 170
特征数: 9

开始训练模型...

模型评估结果

召回率 (Recall):    0.9831 (98.31%)
精确率 (Precision): 0.9667 (96.67%)
准确率 (Accuracy):  0.9824 (98.24%)

混淆矩阵:
              预测负类  预测正类
实际负类         109        2
实际正类           1       58


## 相关问题

### 问题一：逻辑回归的数学原理
1. sigmoid函数有什么重要的数学性质？

2. 逻辑回归使用什么损失函数，为什么不能使用均方误差（MSE）？

**Answer**

1. sigmoid函数的值域是(0,1)，可以将整个实数区间的值都映射到(0,1)之间表示概率。并且sigmoid函数无限次可导，且导数形式简单，便于梯度计算。

2. 逻辑回归使用交叉熵损失函数。当预测值接近0或1时，sigmoid函数的梯度接近0，若使用MSE，则损失函数的梯度也很小，会导致参数更新缓慢。

### 问题二：召回率的理解

在本次作业中，我们引入召回率（Recall）作为模型评估的重要指标。请简单阐明在实验数据集上使用召回率的意义。

**Answer**

本次作业中进行的是对乳腺癌诊断预测的线性回归。对于临床病症检测，应该要尽可能找到所有正确样本不能有遗漏。若不将召回率作为模型评估的指标之一，即使正确率很高，也可能有许多患病样本未被预测出来，导致病人无法及时诊治。


### 问题三：softmax回归的基础概念
1. softmax函数的核心作用是什么？
2. softmax与普通的归一化（如除以总和）有什么本质区别？
3. softmax的计算公式如下，但在实现时可能面临指数过大带来的溢出问题，可以怎么处理？

   $$
   \mathrm{softmax}(\mathbf{x})_i = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}, \quad i=1,\dots ,n
   $$

**Answer**

1. softmax函数将实数向量映射到(0,1)区间中且所有输出和为1，相当于将实数向量转换为概率分布，用于多分类问题的最后一层，输出每个类别的概率

2. a. 即使输入是负数，softmax函数依然能保证输出概率为正，普通的归一化则有可能产生负概率
   b. softmax通过指数运算放大差异，使得最大值对应的概率更加突出
   c. softmax函数处处可导，便于计算梯度与反向传播

3. 可以同时将所有元素减去最大值，使最大元素变为0，其余元素变为负数，以避免指数过大导致溢出。同时由于指数的运算性质，减去后依然在数学上等价。
 


### 选做内容： 分析逻辑回归预测时的阈值（threshold）参数对实验结果的影响

首先测试回归模型在三个不同阈值下的预测结果以及对应评测指标成果，具体如下：

In [79]:
if __name__=='__main__':    
    # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split_manual(x, y, test_size=0.25, random_state=22)
    
    # 特征标准化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # 训练逻辑回归模型
    model = LogisticRegression(learning_rate=0.01, n_iterations=100)
    model.fit(X_train, y_train)
    
    # 预测
    for threshold in [0.3, 0.5, 0.7]:
        y_pred = model.predict(X_test,threshold)

        # 评估
        print("\n" + "=" * 60)
        print("模型评估结果: 阈值 = {:.2f}".format(threshold))
        print("=" * 60)

        recall, precision, accuracy, cm = get_metrics(y_test, y_pred)

        print(f"\n召回率 (Recall):    {recall:.4f} ({recall*100:.2f}%)")
        print(f"精确率 (Precision): {precision:.4f} ({precision*100:.2f}%)")
        print(f"准确率 (Accuracy):  {accuracy:.4f} ({accuracy*100:.2f}%)")


模型评估结果: 阈值 = 0.30

召回率 (Recall):    1.0000 (100.00%)
精确率 (Precision): 0.8806 (88.06%)
准确率 (Accuracy):  0.9529 (95.29%)

模型评估结果: 阈值 = 0.50

召回率 (Recall):    0.9831 (98.31%)
精确率 (Precision): 0.9667 (96.67%)
准确率 (Accuracy):  0.9824 (98.24%)

模型评估结果: 阈值 = 0.70

召回率 (Recall):    0.8644 (86.44%)
精确率 (Precision): 1.0000 (100.00%)
准确率 (Accuracy):  0.9529 (95.29%)


以上结果显示：随着阈值升高，召回率逐步降低，精确率逐步升高，而当阈值居中(0.5)时，准确率最高。

因为阈值较低会倾向于将样本分为正类，导致许多样本把握没那么大的样本会被误分类为正类；反之阈值较高会倾向于将样本分为负类，只有非常有把握的样本会被分类为正类。

应该根据数据集场景选择适当的阈值：
1. 对于如癌症筛查、临床检测的场景下，应该选择较低阈值以避免患病的患者被漏诊

2. 对于如垃圾邮件过滤这类需要高精确度的场景下，应该选择较高阈值以减少误报